# WeatherPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

617

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
#Config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

# Creating lists to store the data

city_name = []
cloudiness = []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []

city_count = 0
set_count = 1


# Loop through the list of cities and perform a request for data on each
for city in cities:
    try:
        city_data = requests.get(query_url + city).json()
        cloudiness.append(city_data['clouds']['all'])
        country.append(city_data['sys']['country'])
        date.append(city_data['dt'])
        humidity.append(city_data['main']['humidity'])
        lat.append(city_data['coord']['lat'])
        lng.append(city_data['coord']['lon'])
        max_temp.append(city_data['main']['temp_max'])
        wind_speed.append(city_data['wind']['speed'])
        if city_count > 48:
            city_count = 1
            set_count += 1
            city_name.append(city)
        else:
            city_count += 1
            city_name.append(city)
    
        print(f'Processing Record {city_count} of set {set_count} | {city_data["name"]}')
    except KeyError as KeyError:
        print('City not found. Skipping...')
print('----------------------------------------------------------------')


Processing Record 1 of set 1 | Port Hawkesbury
Processing Record 2 of set 1 | Saint Paul Harbor
Processing Record 3 of set 1 | Kerema
Processing Record 4 of set 1 | Khandyga
Processing Record 5 of set 1 | Touros
Processing Record 6 of set 1 | Necochea
Processing Record 7 of set 1 | Qaqortoq
Processing Record 8 of set 1 | Nchelenge
Processing Record 9 of set 1 | Bluff
Processing Record 10 of set 1 | Nanning
Processing Record 11 of set 1 | Beringovskiy
Processing Record 12 of set 1 | Albany
Processing Record 13 of set 1 | Byron Bay
Processing Record 14 of set 1 | Srednekolymsk
Processing Record 15 of set 1 | Saskylakh
City not found. Skipping...
Processing Record 16 of set 1 | Chui
Processing Record 17 of set 1 | Hermanus
Processing Record 18 of set 1 | Cape Town
Processing Record 19 of set 1 | Ushuaia
Processing Record 20 of set 1 | Adrar
Processing Record 21 of set 1 | Provideniya
Processing Record 22 of set 1 | Porto União
Processing Record 23 of set 1 | Vestmanna
Processing Record 24

Processing Record 42 of set 4 | Moussoro
Processing Record 43 of set 4 | Stjørdalshalsen
Processing Record 44 of set 4 | Frekhaug
Processing Record 45 of set 4 | Castro
Processing Record 46 of set 4 | Namatanai
Processing Record 47 of set 4 | Faanui
Processing Record 48 of set 4 | Sitka
Processing Record 49 of set 4 | Kaitangata
Processing Record 1 of set 5 | Taltal
Processing Record 2 of set 5 | Buluang
Processing Record 3 of set 5 | São João da Barra
Processing Record 4 of set 5 | San Patricio
Processing Record 5 of set 5 | Sechelt
Processing Record 6 of set 5 | Kirakira
Processing Record 7 of set 5 | Faya
Processing Record 8 of set 5 | Stokmarknes
Processing Record 9 of set 5 | Worland
Processing Record 10 of set 5 | Khatanga
Processing Record 11 of set 5 | Tiksi
Processing Record 12 of set 5 | Bahía Blanca
Processing Record 13 of set 5 | Siaya
Processing Record 14 of set 5 | Chirongui
Processing Record 15 of set 5 | Nikolayevka
Processing Record 16 of set 5 | Picos
Processing Recor

Processing Record 30 of set 8 | Foam Lake
Processing Record 31 of set 8 | Saint-François
Processing Record 32 of set 8 | Ahipara
City not found. Skipping...
Processing Record 33 of set 8 | Husavik
Processing Record 34 of set 8 | Kurchum
Processing Record 35 of set 8 | Baykit
Processing Record 36 of set 8 | Estelle
Processing Record 37 of set 8 | Sabha
Processing Record 38 of set 8 | Selma
Processing Record 39 of set 8 | Kinango
Processing Record 40 of set 8 | Ostrovnoy
Processing Record 41 of set 8 | Aklavik
Processing Record 42 of set 8 | Sovetskiy
Processing Record 43 of set 8 | Skellefteå
Processing Record 44 of set 8 | Kharp
Processing Record 45 of set 8 | Andijan
Processing Record 46 of set 8 | Vila Franca do Campo
Processing Record 47 of set 8 | Mindelo
Processing Record 48 of set 8 | Karla
Processing Record 49 of set 8 | Urucará
Processing Record 1 of set 9 | Ponta Delgada
Processing Record 2 of set 9 | Thompson
Processing Record 3 of set 9 | Bang Saphan
Processing Record 4 of s

Processing Record 21 of set 12 | Sheridan
City not found. Skipping...
Processing Record 22 of set 12 | Bella Union
Processing Record 23 of set 12 | Mattru
Processing Record 24 of set 12 | Tilichiki
Processing Record 25 of set 12 | Calama
----------------------------------------------------------------


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
data_dict = {
    "City":city_name,
    "Cloudiness":cloudiness,
    "Country":country,
    "Date":date,
    "Humidity":humidity,
    "Lat":lat,
    "Lng":lng,
    "Max Temp":max_temp,
    "Wind Speed":wind_speed
}

city_data_df = pd.DataFrame(data_dict)

#city_data_df.head()

In [ ]:
city_data_df.to_csv(output_data_file, encoding="utf-8", index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression